# Experiment 01: Airline dataset

In this notebook we are going to perform a prediction experiment with XGBoost and LightGBM using the Airline dataset. We are also going to analyze if the dataset has [concept drift](https://en.wikipedia.org/wiki/Concept_drift). 

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.

The machine we used was an [Azure NV24 VM](https://azure.microsoft.com/en-gb/blog/azure-n-series-general-availability-on-december-1/), which has [NVIDIA M60](http://www.nvidia.com/object/tesla-m60.html) GPUs. Its operating system is Ubuntu 16.04.

For both XGBoost and LightGBM we compiled from source, to get the last improvements. In XGboost we used the commit `6776292951565c8cd72e69afd9d94de1474f00c0` of May 26th. For LightGBM we used the commit `73968a96829e212b333c88cd44725c8c39c03ad1` of June 2nd. To get these versions and replicate our experiments:
```python
git clone --recursive *url_of_library*
git checkout *oldcommit*
git submodule update --recursive
```

In [2]:
import json
import sys
import warnings

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pkg_resources
from IPython.display import SVG, display
from bokeh.io import export_svgs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.loaders import load_airline
from libs.notebook_memory_management import start_watching_memory
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score, precision_score,
                             recall_score)
from libs.metrics import classification_metrics_binary, classification_metrics_binary_prob, binarize_prediction
from toolz import curry
from xgboost import XGBRegressor

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

warnings.filterwarnings("ignore", category=DeprecationWarning)

System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


The dataset we are going to use in this notebook is huge, therefore we want to monitor the memory consumption. 

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
start_watching_memory()

In [4] used 12.4961 MiB RAM in 24.42s, total RAM usage 271.16 MiB


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [5]:
%%time
df_plane = load_airline()
print(df_plane.shape)

INFO:libs.loaders:MOUNT_POINT not found in environment. Defaulting to /fileshare


(115069017, 14)
CPU times: user 1min 35s, sys: 14.6 s, total: 1min 50s
Wall time: 4min 28s
In [5] used 21994.1484 MiB RAM in 269.00s, total RAM usage 22265.31 MiB


In [6]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [6] used 0.5195 MiB RAM in 0.12s, total RAM usage 22265.83 MiB


The first step is to convert the categorical features to numeric features.

In [7]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 44s, sys: 9.72 s, total: 1min 54s
Wall time: 1min 52s
In [7] used 5270.2930 MiB RAM in 112.72s, total RAM usage 27536.12 MiB


In [8]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [8] used 0.0078 MiB RAM in 0.12s, total RAM usage 27536.13 MiB


In [9]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')

CPU times: user 56.8 s, sys: 7.04 s, total: 1min 3s
Wall time: 1min 2s
In [9] used 12290.7852 MiB RAM in 62.91s, total RAM usage 39826.92 MiB


In [10]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [10] used 0.0039 MiB RAM in 0.12s, total RAM usage 39826.92 MiB


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [11]:
df_plane_numeric = df_plane_numeric.apply(lambda x: x.astype('int16'))

In [11] used 3072.4219 MiB RAM in 12.96s, total RAM usage 42899.34 MiB


In [12]:
%%time
df_plane_numeric['ArrDelayBinary'] = 1*(df_plane_numeric['ArrDelay'] > 0)

CPU times: user 324 ms, sys: 408 ms, total: 732 ms
Wall time: 729 ms
In [12] used 877.9727 MiB RAM in 0.83s, total RAM usage 43777.32 MiB


In [13]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [13] used 0.0000 MiB RAM in 0.12s, total RAM usage 43777.32 MiB


Once the features are prepared, let's split the dataset into train and test set. We won't use validation for this example (however, you can try to add it).

In [14]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [14] used 0.0000 MiB RAM in 0.10s, total RAM usage 43777.32 MiB


In [15]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric, val_size=0, test_size=0.2)
print(train.shape)
print(validate.shape)
print(test.shape)

(92055213, 15)
(0, 15)
(23013804, 15)
CPU times: user 38.4 s, sys: 3.3 s, total: 41.7 s
Wall time: 41 s
In [15] used 4653.2656 MiB RAM in 41.09s, total RAM usage 48430.58 MiB


In [16]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [16] used 0.0000 MiB RAM in 0.11s, total RAM usage 48430.58 MiB


In [17]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)

CPU times: user 872 ms, sys: 600 ms, total: 1.47 s
Wall time: 1.47 s
In [17] used 2853.0195 MiB RAM in 1.57s, total RAM usage 51283.60 MiB


In [18]:
del train, validate, test

In [18] used 0.2422 MiB RAM in 0.10s, total RAM usage 51283.84 MiB


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [19]:
results_dict = dict()
num_rounds = 200

In [19] used 0.0000 MiB RAM in 0.10s, total RAM usage 51283.84 MiB


In [20]:
number_processors = get_number_processors()
print(number_processors)

24
In [20] used 0.0000 MiB RAM in 0.10s, total RAM usage 51283.84 MiB


Let's start with the XGBoost classifier.

In [ ]:
xgb_clf_pipeline = XGBRegressor(max_depth=8,
                                n_estimators=num_rounds,
                                min_child_weight=30,
                                learning_rate=0.1,
                                scale_pos_weight=2,
                                gamma=0.1,
                                reg_lambda=1,
                                subsample=1,
                                n_jobs=number_processors,
                                random_state=77)

In [21] used 0.0000 MiB RAM in 0.10s, total RAM usage 51283.84 MiB


In [ ]:
with Timer() as t:
    xgb_clf_pipeline.fit(X_train, y_train)

In [ ]:
results_dict['xgb']={
    'train_time': t.interval
}

Training XGBoost model with leave-wise growth

In [ ]:
xgb_hist_clf_pipeline = XGBRegressor(max_depth=0,
                                    max_leaves=255,
                                    n_estimators=num_rounds,
                                    min_child_weight=30,
                                    learning_rate=0.1,
                                    scale_pos_weight=2,
                                    gamma=0.1,
                                    reg_lambda=1,
                                    subsample=1,
                                    grow_policy='lossguide',
                                    tree_method='hist',
                                    n_jobs=number_processors,
                                    random_state=77)

In [ ]:
with Timer() as t:
    xgb_hist_clf_pipeline.fit(X_train, y_train)

In [ ]:
results_dict['xgb_hist']={
    'train_time': t.interval
}

Training LightGBM model

In [ ]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                  n_estimators=num_rounds,
                                  min_child_weight=30,
                                  learning_rate=0.1,
                                  scale_pos_weight=2,
                                  min_split_gain=0.1,
                                  reg_lambda=1,
                                  subsample=1,
                                  nthread=number_processors,
                                  seed=77)

In [ ]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train)

In [ ]:
results_dict['lgbm']={
    'train_time': t.interval
}

As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [ ]:
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [ ]:
results_dict['xgb']['test_time'] = t.interval

In [ ]:
with Timer() as t:
    y_prob_xgb_hist = np.clip(xgb_hist_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [ ]:
results_dict['xgb_hist']['test_time'] = t.interval

In [ ]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [ ]:
results_dict['lgbm']['test_time'] = t.interval

### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [ ]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_xgb_hist = binarize_prediction(y_prob_xgb_hist)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)

In [ ]:
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)

In [ ]:
results_dict['xgb']['performance'] = report_xgb

In [ ]:
report_xgb_hist = classification_metrics_binary(y_test, y_pred_xgb_hist)
report2_xgb_hist = classification_metrics_binary_prob(y_test, y_prob_xgb_hist)
report_xgb_hist.update(report2_xgb_hist)

In [ ]:
results_dict['xgb_hist']['performance'] = report_xgb_hist

In [ ]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)

In [ ]:
results_dict['lgbm']['performance'] = report_lgbm

In [ ]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

In [ ]:
del xgb_clf_pipeline, xgb_hist_clf_pipeline, lgbm_clf_pipeline, X_train, X_test, X_val

# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [ ]:
initial_year = 1987
num_ini = 5

In [ ]:
def generate_subset_by_year(df, year_ini, year_end):
    return df[df['Year'].isin(range(year_ini, year_end))]

In [ ]:
%%time
subset_base = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + num_ini)
print(subset_base.shape)

In [ ]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_base.index)]
print(rest_df.shape)

### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [ ]:
X_train, y_train = generate_feables(subset_base)
del(subset_base)

In [ ]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [ ]:
%%time
clf.fit(X_train, y_train)

### Prediction

In [ ]:
@curry
def predict_accuracy(clf, test_df):
    X_test, y_test = generate_feables(test_df)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [ ]:
%%time
accuracy_series = rest_df.groupby('Year').apply(predict_accuracy(clf))
print(accuracy_series)

From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [ ]:
new_init = 15

In [ ]:
%%time
subset_retrain = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + new_init) 
print(subset_retrain.shape)

In [ ]:
X_train, y_train = generate_feables(subset_retrain)

In [ ]:
clf_retrain = LGBMClassifier(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=42)

In [ ]:
%%time
clf_retrain.fit(X_train, y_train)

### Prediction

In [ ]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_retrain.index)]
print(rest_df.shape)

In [ ]:
%%time
accuracy_retrain = rest_df.groupby('Year').apply(predict_accuracy(clf_retrain))
print(accuracy_retrain)

### Plot

In [ ]:
def plot_metrics(metric1, metric2, legend1=None, legend2=None, x_label=None, y_label=None):
    lists = sorted(metric1.items()) 
    x, y = zip(*lists) 
    fig, ax = plt.subplots()
    ax.plot(x, y, label=legend1, color='#5975a4')
    lists2 = sorted(metric2.items()) 
    x2, y2 = zip(*lists2) 
    ax.plot(x2, y2, label=legend2, color='#5f9e6f')
    legend = ax.legend(loc=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()
    return ax

In [ ]:
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool

In [ ]:
data_cds = ColumnDataSource(pd.DataFrame({
    'train_acc': accuracy_series,
    'retrain_acc': accuracy_retrain
}))

In [ ]:
# Airline Retrain Results
p = figure(y_axis_label='Accuracy', plot_width=700, plot_height=350, tools="pan,wheel_zoom,box_zoom,reset")
l1 = p.line('Year', 'train_acc', legend=' Train AUC', line_color="#5975a4", source=data_cds, line_width=6, line_cap="round")
p.line('Year', 'retrain_acc', legend=' Retrain AUC', line_color="#a1bae3", source=data_cds, line_width=6, line_cap="round")
l1_hover = HoverTool(renderers=[l1], tooltips=[( 'Train',  '@{train_acc}{0.4f}' ), ( 'Retrain',  '@{retrain_acc}{0.4f}' )], mode='vline')
p.add_tools(l1_hover)
show(p)

In [ ]:
# SVG: Airline Retrain Results
# Save the plot and display it so that is is visible on Github
p.output_backend = "svg"
export_svgs(p, filename="airline_retrain.svg")
display(SVG('airline_retrain.svg'))

As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.